<a href="https://colab.research.google.com/github/AlexKressner/Business_Intelligence/blob/main/ML_Fallstudie_Online_Barber_Shop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study "Online Barber Shop"
Sie finden unter `./Daten/Barber/` den Ausschnitt eines Datensatzes einer **Direct-to-Consumer (D2C) Brand**, die über einen Onlineshop **Rasierer** und **Rasierzubehör** verkauft. Für das Unternehmen ist es enorm wichtig, dass es die voraussichtlichen **Absätze der nächsten 21 Tage** kennt, da es Waren im Voraus bei seinen Lieferanten bestellen muss. Je genauer die Absatzprognose, desto weniger Stockouts bzw. unnötige Bestände hat das Unternehmen und desto besser die Marge.

Bisher ist es dem Unternehmen nicht gelungen, gute Prognosen zu erstellen. Um dies zu ändern, werden Sie nun beauftragt, eine Prognose mithilfe von ML-Algorithmen anzufertigen. Dazu stellt Ihnen das Unternehmen einen ersten Datensatz aus seinem Shopsystem zur Verfügung. In diesem sind **3 Produkte** enthalten (Alaunstein, Bartbalm und ein Rasierer) und Sie sollen für diese jeweils eine Absatzprogonse erstellen. Bitte orientieren Sie sich bei der Bearbeitung der Fallstudie an der in diesem Notebook vorgeschlagenen Struktur und greifen Sie auf die Notebooks zurück, die wir bereits in der Vorlesung gemeinsam bearbeitet haben.

Die Struktur Ihres Notebooks sollte dabei wie folgt aussehen und u.a. die folgenden Fragen beantworten:

1. **Daten laden & aufbereiten**. Wichtig: Nachdem Sie die Daten geladen haben, müssen Sie diese zunächst auf Tage und je Produkt aggregieren.
2. **Datenexploration**, d.h. verschaffen Sie sich einen Überblick zum Datensatz. Sie sollten folgende Fragen beantworten:
  - Wie viele Absätze wurden über welchen Zeitraum je Produkt dokumentiert?
  - Was ist der Gesamtumsatz je Produkt?
  - Wie sind die Produktverkäufe im Zeitverlauf? Visualisieren Sie beispielweise die monatlichen/wöchentlichen Verkaufsmengen. Dies ist wichtig, um Trends oder Saisonalität in Zeitreihen zu erkennen.
  - Wie sind die durchschnittlichen Absätze pro Wochentag?
3. **Feature Engineering**
  - Entwickeln Sie basirend auf Ihrer Datenexploration nützliche Features für ein Vorhersagemodell.
  - Stellen Sie sich die Frage, mit welchen Features Sie Saisonalitäten abbilden können. Gibt es Monate in den die Absätze deutlich höher sind? Falls ja, sollten Sie ein Features einführen, das den Monat (Jan, Feb, ..., Dez) abbildet!
  - Bilden Sie autoregressive Features, z.B. könnte es Sinn ergeben zur Progonse des Absatzes an einem Tag (z.B. 15.03.) den Absatz des Vormonats zum gleichen Datum (15.02.) heranzuziehen.
  - Können Sie aus den zur Verfügung stehenden Daten den Stückpreis als Feature ableiten?
4. **Prognosemodell trainieren und bewerten**
  - Trainieren Sie ein globales Prognosemodell auf allen Daten und bewerten Sie dieses.
  - Trainieren Sie drei lokale Prognosemodelle (für Alaunstein, Bartbalm & Rasierer) und überprüfen Sie, ob die Prognosegüte verbessert wird.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r'Daten\Barber\barber_shopify_data.csv', sep=';', parse_dates=['Datum'])

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.drop(columns='Bestellung', inplace=True)

In [ ]:
df.head()

In [ ]:
df.rename({'Versand Land': 'Versand_Land'}, inplace=True, axis=1)

In [ ]:
for col in df.columns:
    df.rename({col: col.lower()}, inplace=True, axis=1)
df.head()

In [ ]:
df_agg = df.groupby(['datum', 'produkt']).agg({'nettomenge': 'sum', 'bruttoumsatz': 'sum'}).reset_index()

In [ ]:
def get_agg_for_col(df: pd.DataFrame, group_col: list[str] | str, agg_col: str, agg_func: str = 'sum', sorted: bool = False, ascending: bool = False) -> pd.DataFrame:
    if isinstance(group_col, str):
        group_col = [group_col]
    if agg_func == 'sum':
        agg_result = df.groupby(group_col)[agg_col].sum()
    elif agg_func == 'mean':
        agg_result = df.groupby(group_col)[agg_col].mean()
    elif agg_func == 'count':
        agg_result = df.groupby(group_col)[agg_col].count()
    elif agg_func == 'min':
        agg_result = df.groupby(group_col)[agg_col].min()
    elif agg_func == 'max':
        agg_result = df.groupby(group_col)[agg_col].max()
    else:
        raise ValueError(f"Invalid aggregation function: {agg_func}")
    
    agg_result = agg_result.reset_index()
    
    if sorted:
        agg_result = agg_result.sort_values(by=agg_col, ascending=ascending)
    
    return agg_result


In [ ]:
df_agg.head()

In [ ]:
agg_cols = ['nettomenge', 'bruttoumsatz']
for col in agg_cols:
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=get_agg_for_col(df, ['datum', 'produkt'], col ), x='datum', y=col, hue='produkt')
    plt.show()

In [ ]:
for col in ['nettomenge', 'bruttoumsatz']:
    plt.figure(figsize=(10, 6))
    plt.title(f'Summe {col} pro Produkt')
    sns.barplot(data=get_agg_for_col(df_agg, 'produkt', 'nettomenge', sorted=True), x='produkt', y='nettomenge')
    plt.show()


In [ ]:
df_agg['year'] = df_agg['datum'].dt.year
df_agg['month'] = df_agg['datum'].dt.month
df_agg['day'] = df_agg['datum'].dt.day
df_agg['week'] = df_agg['datum'].dt.isocalendar().week
df_agg['weekday'] = df_agg['datum'].dt.weekday
df_agg['day_of_year'] = df_agg['datum'].dt.dayofyear
df_agg['quarter'] = df_agg['datum'].dt.quarter
df_agg['day_name'] = df_agg['datum'].dt.day_name()

In [ ]:
df_agg.head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=get_agg_for_col(df_agg, ['day_name', 'produkt'], 'nettomenge', agg_func='mean'), x='day_name', y='nettomenge', hue='produkt')
plt.show()